# Prova 2

Parabéns! Após um bem-sucedido mestrado em métodos matemáticos, vocês foram escolhidos como quants
do fundo Short Term INtelliGence (STING).
O fundo está no Brasil, funding e benchmark são o CDI; as vols são offshore mas pode usar como se fosse
onshore.

De posse de uma série histórica de preços, responda as seguintes questões:

#### Exercício 1:
Como estimar o risco dos portfolios do fundo a partir das séries de preços? Aponte as diferenças entre como
estimar o risco entre os diferentes tipos de séries. Assuma que esse cálculo está sendo feito em 31-Mar-2016.

Primeiro dividi cada conjunto de informações do arquivo em sheets diferentes e carreguei cada um em dataframe:

In [10]:
import pandas as pd
import numpy as np
import scipy.stats as st

path = 'C:/Users/RUBENS_PC/Google Drive/Mestrado/semestre04/Pricing and Hedging/Pricing - Prova 02/BBergValues_split.xlsx';
xlsx = pd.ExcelFile(path)
usdbrl_df = xlsx.parse('USDBRL')
ptax_df = xlsx.parse('PTAX')
ibovespa_df = xlsx.parse('Ibovespa')
di1Jan21_df = xlsx.parse('DI1Jan21')
cdi_df = xlsx.parse('CDI')
cds_df = xlsx.parse('CDS')
volATM1M_df = xlsx.parse('VolATM1M')
volATM3M_df = xlsx.parse('VolATM3M')
volATM12M_df = xlsx.parse('VolATM12M')
swapPreCDI1M_df = xlsx.parse('SwapPreCDI1M')
swapPreCDI3M_df = xlsx.parse('SwapPreCDI3M')
swapPreCDI12M_df = xlsx.parse('SwapPreCDI12M')
fwd1M_df = xlsx.parse('FWD1M')
fwd3M_df = xlsx.parse('FWD3M')
fwd12M_df = xlsx.parse('FWD12M')


### Exercício 3:
Em 30-Jun-2015 vários traders compraram opções de compra de USD (venda de BRL) ATMF para diferentes
prazos. Use os dados anexos, interpolação linear para spot e forwards, e a interpolação adequada para vols
ATMF (vol forward constante, variância efetiva é a soma das variâncias efetivas) para estimar os resultados
finais e perdas máximas de cada um dos traders.

Preparando os métodos baseados no Black 76:

In [33]:
# Black 76 formula
# phi: 1 = call e -1 = put;
# fwd: forward price
# K: strike
# vol: volatility
# df: discount factor
# t: time to maturity

def black76(phi, fwd, K, vol, t, df):
    efv=t*(vol**2)
    # calculate d1 and d2
    d1=(np.log(fwd/K)+(efv/2))/(np.sqrt(efv))
    d2=(np.log(fwd/K)-(efv/2))/(np.sqrt(efv))
    # calculate N(d1) and N(d2)
    Nd1=st.norm.cdf(phi*d1)
    Nd2=st.norm.cdf(phi*d2)
    # calculate premium
    pr=phi*(fwd*Nd1 - K*Nd2)*df
    # calculate delta
    de=phi*Nd1*df
        
    return [pr, de]

call=1
put=-1

(a) O primeiro comprou uma opção de 1m e carregou sem hedge até o final.

Vamos calcular o preço e o delta da call de 1M no dia 30-Jun-2015:

In [34]:
fxfwd = fwd1M_df.loc[fwd1M_df['Date'] == '30-Jun-2015']['PX_LAST'].values[0] # Forward 1 mês (On)
volATM1M = volATM1M_df.loc[volATM1M_df['Date'] == '30-Jun-2015']['PX_LAST'].values[0] # Vol Implícita ATM 1 mês (Off)
swapPreCDI1M = swapPreCDI1M_df.loc[swapPreCDI1M_df['Date'] == '30-Jun-2015']['PX_LAST'].values[0] # Swap Pré-CDI 1 mês

K = fxfwd # ATMF
vol = volATM1M/100
t = 30/365
df = 1/((1 + (swapPreCDI1M/100))**t)

fxfwd, K, vol, t, df

(3.1364000000000001,
 3.1364000000000001,
 0.16111999999999999,
 0.0821917808219178,
 0.98952100449935765)

In [35]:
black76D0 = black76(call, fxfwd, K, vol, t, df)
black76D0

[0.057186243352910809, 0.50387704404168732]

##### Calculando o preço e o delta da opção para o dia 1-Jul-2015.

Primeiro devemos encontrar o FWD para o vencimento da opção (30-Jul-2015), interpolando o USDBRL do dia 1-Jul-2015 com o FWD de 1 mês também visto no dia 1-Jul-2015:

In [36]:
def linearInterpolation(x0, y0, x1, y1, x):
    y = y0 + (y1 - y0) * (x - x0) / (x1 - x0)
    return y